In [1]:
from faker import Faker

import faker_commerce
import mysql.connector as connector
import random
import datetime

In [ ]:
HOST = "localhost"
USERNAME = "root"
PASSWORD = "ptit"
DATABASE = "e_commerce"
PORT = 3307

In [ ]:
random.seed(1000)

Faker.seed(1000)
fake = Faker()
fake.add_provider(faker_commerce.Provider)

conn = connector.connect(host=HOST, user=USERNAME, password=PASSWORD, database=DATABASE, port=PORT)
curr = conn.cursor()

In [39]:
# Insert country
no_countries = 50
country_query = "INSERT INTO countries(country_name) VALUES (%s)"
try:
    for i in range(no_countries):
        curr.execute(country_query, [fake.country()])
        conn.commit()
except Exception as e:
    print(e)

In [40]:
# Insert address
no_addresses = 1000
address_query = "INSERT INTO " \
                "addresses(address_line_1, address_line_2, street, city, country_id)" \
                "VALUES (%s, %s, %s, %s, %s)"

try:
    for i in range(no_addresses):
        curr.execute(address_query, [fake.street_address(), fake.secondary_address(), fake.street_name(), fake.city(), random.randint(1, no_countries)])
        conn.commit()
except Exception as e:
    print(e)

In [41]:
# insert customer
no_customers = 10000
customer_query = "INSERT INTO " \
                 "customers(first_name, last_name, phone_number, date_of_birth, email, gender, password)" \
                 "VALUES(%s, %s, %s, %s, %s, %s, %s)"
gender = ["male", "female"]
digits = "".join(random.choices("0123456789", k=9))
try:
    for i in range(no_customers):
        val = [fake.first_name(), fake.last_name(), fake.country_calling_code() + digits, fake.date_of_birth(minimum_age=18, maximum_age=70), fake.email(), gender[random.randint(0, 1)], fake.text(max_nb_chars=12)]
        curr.execute(customer_query, val)
        conn.commit()
except Exception as e:
    print(e)

In [42]:
# customer_address
customer_address_query = "INSERT INTO " \
                            "customer_address(customer_id, address_id) " \
                            "VALUES(%s, %s)"
try:
    for i in range(1, no_customers+1):
        curr.execute(customer_address_query, [i, random.randint(1, no_addresses)])
        conn.commit()
except Exception as e:
    print(e)

In [43]:
# order
no_orders = 20000
order_query = "INSERT INTO orders(customer_id, order_date, total_amount, status) VALUES (%s, %s, %s, %s)"
order_statuses = [
    "PENDING",     # Chờ xử lý
    "CONFIRMED",   # Đã xác nhận
    "PROCESSING",  # Đang xử lý
    "SHIPPED",     # Đã giao cho đơn vị vận chuyển
    "DELIVERED",   # Đã giao hàng thành công
    "CANCELLED",   # Đã hủy
    "RETURNED",    # Khách hoàn trả hàng
    "REFUNDED"     # Đã hoàn tiền
]
start_date = datetime.date(2015, 1, 1)
end_date = datetime.date.today()
for i in range(no_orders):
    val = [random.randint(1, no_customers), fake.date_between(start_date, end_date), round(random.uniform(1,100000),2), order_statuses[random.randint(0, len(order_statuses)-1)]]
    curr.execute(order_query, val)
    conn.commit()

In [44]:
# supplier
no_suppliers = 1000
supplier_query = "INSERT INTO suppliers(supplier_name, country) VALUES (%s, %s)"
try:
    for i in range(no_suppliers):
        val = [fake.company(), fake.country()]
        # print(val[1])
        curr.execute(supplier_query, val)
        conn.commit()
except Exception as e:
    raise(str(e))

In [45]:
# Product
no_products = 100000
product_query = "INSERT INTO products(product_name, price, quantity_in_stock, image_url, supplier_id) VALUES (%s, %s, %s, %s, %s)"
for i in range(no_products):
    val = [fake.ecommerce_name(), round(random.uniform(1,10000), 2), random.randint(0, 1000), fake.image_url(), random.randint(1, no_suppliers)]
    curr.execute(product_query, val)
    conn.commit()

In [46]:
# order_product
order_product_query = "INSERT INTO order_product(product_id, order_id, number_of_product) VALUES(%s, %s, %s)"
for _ in range(no_orders):
    val = [random.randint(1, no_products), random.randint(1, no_orders), random.randint(1,5)]
    curr.execute(order_product_query, val)
    conn.commit()

In [47]:
# Reviews
no_reviews = 10000
review_query = "INSERT INTO reviews(product_id, star, review) VALUES (%s, %s, %s)"
for _ in range(no_reviews):
    val = [random.randint(1, no_products), random.randint(1,5), fake.text(max_nb_chars=255)]
    curr.execute(review_query, val)
    conn.commit()

In [48]:
# Category
no_categories = 5000
category_tags = [
    "Electronics",
    "Fashion",
    "Home & Garden",
    "Beauty",
    "Sports",
    "Automotive",
    "Toys",
    "Books",
    "Health",
    "Office Supplies"
]


category_query = "INSERT INTO categories(category_name, description, tag) VALUES(%s, %s, %s)"
for _ in range(no_categories):
    val = [fake.ecommerce_category(), fake.text(max_nb_chars=255), random.choice(category_tags)]
    curr.execute(category_query, val)
    conn.commit()

In [49]:
# product_category
product_category_query = "INSERT INTO product_category(product_id, category_id) VALUES (%s, %s)"
for _ in range(10000):
    val = [random.randint(1, no_products), random.randint(1, no_categories)]
    curr.execute(product_category_query, val)
    conn.commit()